In [13]:
import numpy as np
import pandas as pd

# Manually deleted first row of the excel sheet that was messing up the column names
df_edu = pd.read_csv('E:\\Career\\Projects\\Data-Driven GDP Analysis\\data\\education.csv', encoding='ISO-8859-1')

df_edu

,Region/Country/Area,Unnamed: 1,Year,Series,Value,Footnotes,Source
0,1,"Total, all countries or areas",2005,Students enrolled in primary education (thousa...,"678,907",Estimate.,"United Nations Educational, Scientific and Cul..."
1,1,"Total, all countries or areas",2005,Gross enrollment ratio - Primary (male),104.5,Estimate.,"United Nations Educational, Scientific and Cul..."
2,1,"Total, all countries or areas",2005,Gross enrollment ratio - Primary (female),99.7,NaN,"United Nations Educational, Scientific and Cul..."
3,1,"Total, all countries or areas",2005,Students enrolled in lower secondary education...,"309,665",NaN,"United Nations Educational, Scientific and Cul..."
4,1,"Total, all countries or areas",2005,Gross enrollment ratio - Lower secondary level...,80.7,NaN,"United Nations Educational, Scientific and Cul..."
...,...,...,...,...,...,...,...
7525,722,SIDS,2020,Gross enrollment ratio - Lower secondary level...,76.5,Estimate.,"United Nations Educational, Scientific and Cul..."
7526,722,SIDS,2020,Gross enrollment ratio - Lower secondary level...,74.8,Estimate.,"United Nations Educational, Scientific and Cul..."
7527,722,SIDS,2020,Students enrolled in upper secondary education...,"1,890",Estimate.,"United Nations Educational, Scientific and Cul..."
7528,722,SIDS,2020,Gross enrollment ratio - Upper secondary level...,56.1,Estimate.,"United Nations Educational, Scientific and Cul..."


In [14]:
# Doing some columns fixes

df_edu = df_edu.drop(columns = ['Footnotes', 'Source', 'Region/Country/Area']).rename(columns = {'Unnamed: 1':'country',
                                                                                                     'Year': 'year'})

df_edu.head()

,country,year,Series,Value
0,"Total, all countries or areas",2005,Students enrolled in primary education (thousa...,"678,907"
1,"Total, all countries or areas",2005,Gross enrollment ratio - Primary (male),104.5
2,"Total, all countries or areas",2005,Gross enrollment ratio - Primary (female),99.7
3,"Total, all countries or areas",2005,Students enrolled in lower secondary education...,"309,665"
4,"Total, all countries or areas",2005,Gross enrollment ratio - Lower secondary level...,80.7


In [15]:
# Removing the region observations, all before Afghanistan

afghan_index = df_edu[df_edu['country'] == 'Afghanistan'].index[0] # first index that matches the condition
df_edu = df_edu.loc[afghan_index: ]

df_edu

,country,year,Series,Value
468,Afghanistan,2005,Students enrolled in primary education (thousa...,"4,319"
469,Afghanistan,2005,Gross enrollment ratio - Primary (male),123.1
470,Afghanistan,2005,Gross enrollment ratio - Primary (female),71.8
471,Afghanistan,2005,Students enrolled in lower secondary education...,461
472,Afghanistan,2005,Gross enrollment ratio - Lower secondary level...,36.9
...,...,...,...,...
7525,SIDS,2020,Gross enrollment ratio - Lower secondary level...,76.5
7526,SIDS,2020,Gross enrollment ratio - Lower secondary level...,74.8
7527,SIDS,2020,Students enrolled in upper secondary education...,"1,890"
7528,SIDS,2020,Gross enrollment ratio - Upper secondary level...,56.1


In [16]:
# SIDS stands for "Small Island Developing States". We will remove these type of observations for now by removing
# observations after Zimbabwe

zimbab_last_index = df_edu[df_edu['country'] == 'Zimbabwe'].index[-1] #-1 selects the last element from the index array
df_edu = df_edu.loc[:zimbab_last_index] # Pandas slicing using `.loc` is inclusive therefore no need for the + 1

df_edu

,country,year,Series,Value
468,Afghanistan,2005,Students enrolled in primary education (thousa...,"4,319"
469,Afghanistan,2005,Gross enrollment ratio - Primary (male),123.1
470,Afghanistan,2005,Gross enrollment ratio - Primary (female),71.8
471,Afghanistan,2005,Students enrolled in lower secondary education...,461
472,Afghanistan,2005,Gross enrollment ratio - Lower secondary level...,36.9
...,...,...,...,...
7417,Zimbabwe,2015,Gross enrollment ratio - Primary (male),104.9
7418,Zimbabwe,2015,Gross enrollment ratio - Primary (female),102.8
7419,Zimbabwe,2021,Students enrolled in primary education (thousa...,"2,899"
7420,Zimbabwe,2021,Gross enrollment ratio - Primary (male),95.9


In [17]:
# Making the `Value` column a numerical double
df_edu = df_edu.copy()

df_edu.loc[:, 'Value'] = df_edu['Value'].str.replace(',', '').astype(float)
df_edu.dtypes

country     object
year         int64
Series      object
Value      float64
dtype: object

In [18]:
# Checking NAs

df_edu.isna().sum()

country    0
year       0
Series     0
Value      0
dtype: int64

## Pivoting the `Series` column

In [19]:
df_edu['Series'].unique()

array(['Students enrolled in primary education (thousands)',
       'Gross enrollment ratio - Primary (male)',
       'Gross enrollment ratio - Primary (female)',
       'Students enrolled in lower secondary education (thousands)',
       'Gross enrollment ratio - Lower secondary level (male)',
       'Gross enrollment ratio - Lower secondary level (female)',
       'Students enrolled in upper secondary education (thousands)',
       'Gross enrollment ratio - Upper secondary level (male)',
       'Gross enrollment ratio - Upper secondary level (female)'],
      dtype=object)

In [20]:
# Renaming to prepare for columns

df_edu['Series'].replace({
    'Students enrolled in primary education (thousands)':           'primary_total_enrolled_thousands',
    'Gross enrollment ratio - Primary (male)':                      'primary_enrollment_ratio_male',
    'Gross enrollment ratio - Primary (female)':                    'primary_enrollment_ratio_female',
    'Students enrolled in lower secondary education (thousands)':   'lower_secondary_total_enrolled_thousands',
    'Gross enrollment ratio - Lower secondary level (male)':        'lower_secondary_enrollment_ratio_male',
    'Gross enrollment ratio - Lower secondary level (female)':      'lower_secondary_enrollment_ratio_female',
    'Students enrolled in upper secondary education (thousands)':   'upper_secondary_total_enrolled_thousands',
    'Gross enrollment ratio - Upper secondary level (male)':        'upper_secondary_enrollment_ratio_male',
    'Gross enrollment ratio - Upper secondary level (female)':      'upper_secondary_enrollment_ratio_female'
},
    inplace = True)

df_edu

,country,year,Series,Value
468,Afghanistan,2005,primary_total_enrolled_thousands,4319.0
469,Afghanistan,2005,primary_enrollment_ratio_male,123.1
470,Afghanistan,2005,primary_enrollment_ratio_female,71.8
471,Afghanistan,2005,lower_secondary_total_enrolled_thousands,461.0
472,Afghanistan,2005,lower_secondary_enrollment_ratio_male,36.9
...,...,...,...,...
7417,Zimbabwe,2015,primary_enrollment_ratio_male,104.9
7418,Zimbabwe,2015,primary_enrollment_ratio_female,102.8
7419,Zimbabwe,2021,primary_total_enrolled_thousands,2899.0
7420,Zimbabwe,2021,primary_enrollment_ratio_male,95.9


In [21]:
# Pivoting, 

df_pivot = df_edu.pivot_table(index = ['country', 'year'], columns = 'Series', values = 'Value', aggfunc = 'first').reset_index()

df_edu = df_pivot
df_edu

Series,country,year,lower_secondary_enrollment_ratio_female,lower_secondary_enrollment_ratio_male,lower_secondary_total_enrolled_thousands,primary_enrollment_ratio_female,primary_enrollment_ratio_male,primary_total_enrolled_thousands,upper_secondary_enrollment_ratio_female,upper_secondary_enrollment_ratio_male,upper_secondary_total_enrolled_thousands
0,Afghanistan,2005,12.7,36.9,461.0,71.8,123.1,4319.0,4.9,17.5,190.0
1,Afghanistan,2010,46.4,87.4,1477.0,80.6,118.6,5279.0,17.8,42.7,567.0
2,Afghanistan,2015,45.5,77.9,1669.0,83.5,122.7,6199.0,27.1,52.6,982.0
3,Afghanistan,2018,50.6,86.3,1983.0,82.9,124.2,6545.0,28.5,52.4,1081.0
4,Afghanistan,2019,52.2,87.6,2067.0,85.4,127.1,6778.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
900,Zambia,2017,59.8,61.7,511.0,99.9,97.5,3285.0,NaN,NaN,NaN
901,Zimbabwe,2003,57.3,61.4,387.0,NaN,NaN,NaN,NaN,NaN,NaN
902,Zimbabwe,2013,75.2,72.8,467.0,108.7,111.1,2663.0,38.9,43.2,491.0
903,Zimbabwe,2015,NaN,NaN,NaN,102.8,104.9,2658.0,NaN,NaN,NaN


## Adding continent column

In [22]:
# Same methodology when cleaning the gdp data

import pycountry_convert as pc

def get_continent(country_name):
    try:
        # Make sure country_name is a string, not a Series or DataFrame
        assert isinstance(country_name, str), f"Input must be str, got {type(country_name)}"
        
        country_code = pc.country_name_to_country_alpha2(country_name)
        continent_code = pc.country_alpha2_to_continent_code(country_code)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except Exception as e:
        print(f"Error processing country: {country_name} - {e}")
        # Return None or you could specify a default value, e.g., 'Unknown'
        return None

# Apply get_continent function row-wise using apply with axis=1
df_test = df_edu.copy()
df_test['continent'] = df_edu['country'].apply(get_continent)
df_test

Error processing country: Bolivia (Plurin. State of) - "Invalid Country Name: 'Bolivia (Plurin. State of)'"
Error processing country: Bolivia (Plurin. State of) - "Invalid Country Name: 'Bolivia (Plurin. State of)'"
Error processing country: Bolivia (Plurin. State of) - "Invalid Country Name: 'Bolivia (Plurin. State of)'"
Error processing country: Bolivia (Plurin. State of) - "Invalid Country Name: 'Bolivia (Plurin. State of)'"
Error processing country: China, Hong Kong SAR - "Invalid Country Name: 'China, Hong Kong SAR'"
Error processing country: China, Hong Kong SAR - "Invalid Country Name: 'China, Hong Kong SAR'"
Error processing country: China, Hong Kong SAR - "Invalid Country Name: 'China, Hong Kong SAR'"
Error processing country: China, Hong Kong SAR - "Invalid Country Name: 'China, Hong Kong SAR'"
Error processing country: China, Macao SAR - "Invalid Country Name: 'China, Macao SAR'"
Error processing country: China, Macao SAR - "Invalid Country Name: 'China, Macao SAR'"
Error pr

Series,country,year,lower_secondary_enrollment_ratio_female,lower_secondary_enrollment_ratio_male,lower_secondary_total_enrolled_thousands,primary_enrollment_ratio_female,primary_enrollment_ratio_male,primary_total_enrolled_thousands,upper_secondary_enrollment_ratio_female,upper_secondary_enrollment_ratio_male,upper_secondary_total_enrolled_thousands,continent
0,Afghanistan,2005,12.7,36.9,461.0,71.8,123.1,4319.0,4.9,17.5,190.0,Asia
1,Afghanistan,2010,46.4,87.4,1477.0,80.6,118.6,5279.0,17.8,42.7,567.0,Asia
2,Afghanistan,2015,45.5,77.9,1669.0,83.5,122.7,6199.0,27.1,52.6,982.0,Asia
3,Afghanistan,2018,50.6,86.3,1983.0,82.9,124.2,6545.0,28.5,52.4,1081.0,Asia
4,Afghanistan,2019,52.2,87.6,2067.0,85.4,127.1,6778.0,NaN,NaN,NaN,Asia
...,...,...,...,...,...,...,...,...,...,...,...,...
900,Zambia,2017,59.8,61.7,511.0,99.9,97.5,3285.0,NaN,NaN,NaN,Africa
901,Zimbabwe,2003,57.3,61.4,387.0,NaN,NaN,NaN,NaN,NaN,NaN,Africa
902,Zimbabwe,2013,75.2,72.8,467.0,108.7,111.1,2663.0,38.9,43.2,491.0,Africa
903,Zimbabwe,2015,NaN,NaN,NaN,102.8,104.9,2658.0,NaN,NaN,NaN,Africa


In [23]:
# Renaming and Removing Countries

df_edu['country'] = df_edu['country'].replace('Bolivia (Plurin. State of)', 'Bolivia')
df_edu['country'] = df_edu['country'].replace('China, Hong Kong SAR', 'Hong Kong')
df_edu['country'] = df_edu['country'].replace('China, Macao SAR', 'Macao')
df_edu['country'] = df_edu['country'].replace('Côte d\x92Ivoire', 'Ivory Coast')
df_edu['country'] = df_edu['country'].replace("Dem. People's Rep. Korea", 'North Korea')
df_edu['country'] = df_edu['country'].replace("Dem. Rep. of the Congo", 'Democratic Republic of the Congo')
df_edu['country'] = df_edu['country'].replace('Iran (Islamic Republic of)', 'Iran')
df_edu['country'] = df_edu['country'].replace('Micronesia (Fed. States of)', 'Micronesia')
df_edu['country'] = df_edu['country'].replace('Netherlands (Kingdom of the)', 'Netherlands')
df_edu['country'] = df_edu['country'].replace('Republic of Korea', 'South Korea')
df_edu['country'] = df_edu['country'].replace('United Rep. of Tanzania', 'Tanzania')
df_edu['country'] = df_edu['country'].replace('Venezuela (Boliv. Rep. of)', 'Venezuela')

In [24]:
countries_to_remove = ["Lao People's Dem. Rep.", 'Netherlands Antilles [former]', 'Sint Maarten (Dutch part)']
df_removal = df_edu[~df_edu['country'].isin(countries_to_remove)]

df_test = df_removal.copy() # Pandas gets mad when you don't make it explicitly clear that it's a copy
df_test['continent'] = df_test['country'].apply(get_continent)

Error processing country: Saint Vincent & Grenadines - "Invalid Country Name: 'Saint Vincent & Grenadines'"
Error processing country: Saint Vincent & Grenadines - "Invalid Country Name: 'Saint Vincent & Grenadines'"
Error processing country: Saint Vincent & Grenadines - "Invalid Country Name: 'Saint Vincent & Grenadines'"
Error processing country: Saint Vincent & Grenadines - "Invalid Country Name: 'Saint Vincent & Grenadines'"
Error processing country: Saint Vincent & Grenadines - "Invalid Country Name: 'Saint Vincent & Grenadines'"
Error processing country: State of Palestine - "Invalid Country Name: 'State of Palestine'"
Error processing country: State of Palestine - "Invalid Country Name: 'State of Palestine'"
Error processing country: State of Palestine - "Invalid Country Name: 'State of Palestine'"
Error processing country: State of Palestine - "Invalid Country Name: 'State of Palestine'"
Error processing country: Sudan [former] - "Invalid Country Name: 'Sudan [former]'"
Error pr

In [25]:
# Inserting continent for some countries manually
df_test.loc[df_test['country'] == 'State of Palestine', 'continent'] = 'Africa'
df_test.loc[df_test['country'] == 'Kosovo', 'continent'] = 'Europe'
df_test.loc[df_test['country'] == 'Timor-Leste', 'continent'] = 'Asia'
df_test.loc[df_test['country'] == 'Saint Vincent & Grenadines', 'continent'] = 'North America'
df_test.loc[df_test['country'] == 'Zanzibar', 'continent'] = 'Africa'

# Replacing 'Sudan [former]' with Sudan before 2010
df_test.loc[(df_test['country'] == 'Sudan [former]') & (df_test['year'] < 2010), 'country'] = 'Sudan' 
df_test = df_test[df_test['country'] != 'Sudan [former]']
df_test.loc[df_test['country'] == 'Sudan', 'continent'] = 'Africa'

In [26]:
df_test

Series,country,year,lower_secondary_enrollment_ratio_female,lower_secondary_enrollment_ratio_male,lower_secondary_total_enrolled_thousands,primary_enrollment_ratio_female,primary_enrollment_ratio_male,primary_total_enrolled_thousands,upper_secondary_enrollment_ratio_female,upper_secondary_enrollment_ratio_male,upper_secondary_total_enrolled_thousands,continent
0,Afghanistan,2005,12.7,36.9,461.0,71.8,123.1,4319.0,4.9,17.5,190.0,Asia
1,Afghanistan,2010,46.4,87.4,1477.0,80.6,118.6,5279.0,17.8,42.7,567.0,Asia
2,Afghanistan,2015,45.5,77.9,1669.0,83.5,122.7,6199.0,27.1,52.6,982.0,Asia
3,Afghanistan,2018,50.6,86.3,1983.0,82.9,124.2,6545.0,28.5,52.4,1081.0,Asia
4,Afghanistan,2019,52.2,87.6,2067.0,85.4,127.1,6778.0,NaN,NaN,NaN,Asia
...,...,...,...,...,...,...,...,...,...,...,...,...
900,Zambia,2017,59.8,61.7,511.0,99.9,97.5,3285.0,NaN,NaN,NaN,Africa
901,Zimbabwe,2003,57.3,61.4,387.0,NaN,NaN,NaN,NaN,NaN,NaN,Africa
902,Zimbabwe,2013,75.2,72.8,467.0,108.7,111.1,2663.0,38.9,43.2,491.0,Africa
903,Zimbabwe,2015,NaN,NaN,NaN,102.8,104.9,2658.0,NaN,NaN,NaN,Africa


In [28]:
df_test.dtypes

Series
country                                      object
year                                          int64
lower_secondary_enrollment_ratio_female     float64
lower_secondary_enrollment_ratio_male       float64
lower_secondary_total_enrolled_thousands    float64
primary_enrollment_ratio_female             float64
primary_enrollment_ratio_male               float64
primary_total_enrolled_thousands            float64
upper_secondary_enrollment_ratio_female     float64
upper_secondary_enrollment_ratio_male       float64
upper_secondary_total_enrolled_thousands    float64
continent                                    object
dtype: object

In [30]:
df_edu_clean = df_test.copy()

# Creating csv and hierarchical excel file

df_edu_clean.to_csv('E:\\Career\\Projects\\Data-Driven GDP Analysis\\dataframes\\education_df.csv', index = False)

df_edu_hier = df_edu_clean.sort_values(by = ['continent', 'country']).set_index(['continent', 'country', 'year'])
df_edu_hier.to_excel('E:\\Career\\Projects\\Data-Driven GDP Analysis\\dataframes\\education_df_hierarchical.xlsx', index = True)